## kNN implementation for a Classification Problem
The model for kNN is the entire dataset. When a prediction is required for a unseen data instance, the kNN algorithm will search through the training dataset for the k-most similar instances. The prediction attribute of the most similar instances is summarized and returned as the prediction for the unseen data.

The similarity measure is dependent on the type of data. For real-valued data, the Euclidean distance can be used.
Other types of data such as categorical or binary , other distance measures could be used.

In the case of regression problems, the average of the predicted attribute may be returned. In the case of classification problems, the most prevalent class may be returned.

### Handle Data and make them to train and test set

In [1]:
import csv
import random
import numpy as np

In [2]:
def loadDataset(filename, split, trainingSet = [], testSet = []):
    with open(filename) as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset) - 1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

In [3]:
trainingSet = []
testSet = []
a = loadDataset('iris.data.txt', 0.66, trainingSet, testSet)
print(trainingSet[0:3])
print(testSet[0:3])

[[4.9, 3.0, 1.4, 0.2, 'Iris-setosa'], [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa']]
[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'], [4.8, 3.0, 1.4, 0.1, 'Iris-setosa']]


### Similarity
In order to make predictions we need to calculate the similarity between any two given data instances. In this case it is the Euclidean distance which is the square root of the sum of the squared differences between the two array of numbers. 

In [4]:
def euclideanDistance(instance1, instance2, length):  # length : length of the array you want the distance to be calculated
    distance = 0
    for x in range(length):
        distance += np.power((instance1[x] - instance2[x]), 2)
    return np.sqrt(distance)

In [5]:
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
euc_dist = euclideanDistance(data1, data2, 3)
print(euc_dist)

3.4641016151377544


### Neighbors
Now that we have a similarity measure, we can use it to collect the k-most similar instances for a given unseen instance.
So, in principle we are calculating the distances of all instances of the train set with the one instance of test set (unseen set) and selecting a subset with the smallest distance values.

In [6]:
import operator

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance) - 1 # removing the response column from the array
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist)) # a tuple of training set observation and the distance.
    distances.sort(key = operator.itemgetter(1)) # sort the tuple by the value (ascending) as the input arg is 1
    neighbors = []
    for x in range(k): # k-nearest neighbors
        neighbors.append(distances[x][0])  # just the train instance and not the distance
    return neighbors

In [7]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


### Response
Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on these neighbors. We can do it by allowing each neighbor to vote for their class attribute and take majority vote as the prediction.

In [8]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1] # extracting the class value of the neighbors
        if response not in classVotes:
            classVotes[response] = 1
        else:
            classVotes[response] += 1
    sortedVotes = sorted(classVotes.items(), key = operator.itemgetter(1), reverse=True) # descending by values
    return sortedVotes[0][0] # 1st tuple and 1st item which is the response variable (with highest vote)

In [9]:
neighbors = [[2, 2, 2, 'a'], [1, 1, 1, 'a'], [3, 3, 3, 'c']]
response = getResponse(neighbors)
print(response)

a


### Accuracy

In [10]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct / float(len(testSet))) * 100.00

In [11]:
testSet = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [12]:
def main():
    trainingSet = []
    testSet = []
    split = 0.67
    loadDataset('iris.data.txt', split, trainingSet, testSet)
    
    predictions = []
    k = 10
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('Predicted = ' +repr(result)+ ' Actual = ' +repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' +repr(accuracy)+ '%')
main()

Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-setosa' Actual = 'Iris-setosa'
Predicted = 'Iris-versicolor' Actual = 'Iris-versicolor'
Predicted = 'Iris-versicolor' Actual = 'Iris-versicolor'
Predicted = 'Iris-versicolor' Actual = 'Iris-versicolor'
Predicted = 'Iris-versicolor' Actual = 'Iris-versicolor'
Predicted = 'Iris-versicolor' Actual = 'Iris-versicolor'
Predicted = 'Iris-versicolor'